In [62]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder,OneHotEncoder
import pickle

In [63]:
#load pickle file
from tensorflow.keras.models import  load_model
model = load_model("data/model.h5")

In [64]:
with open("data/label_encoder_gender.pkl","rb") as f:
    label_encoder_gen = pickle.load(f)

with open("data/one_hot_encoder_gro.pkl","rb") as f:
    one_hot_encoder_gro = pickle.load(f)

with open("data/scaler.pkl","rb") as f:
    scaler = pickle.load(f)
    

In [81]:
input_data= {
    "CreditScore":600,
    "Geography":"France",
    "Gender":"Male",
    "Age":40,
    "Tenure":3,
    "Balance":60000,
    "NumOfProducts":2,
    "HasCrCard":1,
    "IsActiveMember":1,
    "EstimatedSalary":50000
}

In [82]:
geo_encoded = one_hot_encoder_gro.transform([[input_data["Geography"]]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=one_hot_encoder_gro.get_feature_names_out())
geo_encoded_df

e:\ANN\venv\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [83]:
input_df =pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [84]:
input_df["Gender"] = label_encoder_gen.transform(input_df["Gender"])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [85]:
input_df.drop("Geography",axis=1,inplace=True)

In [86]:
input_df = pd.concat([input_df.reset_index(drop=True),geo_encoded_df],axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [88]:
input_df_scale = scaler.transform(input_df)
input_df_scale 

array([[-0.54007722,  0.91186722,  0.10685843, -0.69164931, -0.26034447,
         0.81311138,  0.64946204,  0.96978946, -0.88017893,  1.00053348,
        -0.57776083, -0.57735027]])

In [89]:
pred = model.predict(input_df_scale)
pred

1/1 [==============================] - 3s 3s/step


array([[0.03896099]], dtype=float32)

In [90]:
pred_proba = pred[0][0]
pred_proba

0.03896099

In [91]:
if pred_proba > .5:
    print("The customer is likely to churn")
else:
    print("The customer is not likely to churn")

The customer is not likely to churn
